Import Dependancies

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("sentiment-emotion-labelled_Dell_tweets.csv")

In [3]:
df.head()

,Unnamed: 0,Datetime,Tweet Id,Text,Username,sentiment,sentiment_score,emotion,emotion_score
0,0,2022-09-30 23:29:15+00:00,1575991191170342912,@Logitech @apple @Google @Microsoft @Dell @Len...,ManjuSreedaran,neutral,0.853283,anticipation,0.587121
1,1,2022-09-30 21:46:35+00:00,1575965354425131008,@MK_habit_addict @official_stier @MortalKombat...,MiKeMcDnet,neutral,0.519470,joy,0.886913
2,2,2022-09-30 21:18:02+00:00,1575958171423752203,"As @CRN celebrates its 40th anniversary, Bob F...",jfollett,positive,0.763791,joy,0.960347
3,3,2022-09-30 20:05:24+00:00,1575939891485032450,@dell your customer service is horrible especi...,daveccarr,negative,0.954023,anger,0.983203
4,4,2022-09-30 20:03:17+00:00,1575939359160750080,@zacokalo @Dell @DellCares @Dell give the man ...,heycamella,neutral,0.529170,anger,0.776124


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24970 entries, 0 to 24969
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       24970 non-null  int64  
 1   Datetime         24970 non-null  object 
 2   Tweet Id         24970 non-null  int64  
 3   Text             24970 non-null  object 
 4   Username         24970 non-null  object 
 5   sentiment        24970 non-null  object 
 6   sentiment_score  24970 non-null  float64
 7   emotion          24970 non-null  object 
 8   emotion_score    24970 non-null  float64
dtypes: float64(2), int64(2), object(5)
memory usage: 1.7+ MB


In [5]:
import string
from nltk.corpus import stopwords

In [6]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(df["Text"])

# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

58135


In [11]:
messages_bow = bow_transformer.transform(df['Text'])

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow)

In [13]:
messages_tfidf = tfidf_transformer.transform(messages_bow)

In [14]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(messages_tfidf, df["sentiment"])

In [15]:
all_predictions = spam_detect_model.predict(messages_tfidf)
print(all_predictions)

['neutral' 'negative' 'positive' ... 'neutral' 'neutral' 'negative']


In [24]:
print(len(np.unique(all_predictions)))
print(len(np.unique(df["sentiment"])))

3
3


In [29]:
np.mean(all_predictions == df["sentiment"])

0.7992791349619544

In [25]:
from sklearn.metrics import classification_report, confusion_matrix

In [27]:
confusion_matrix(df['sentiment'], all_predictions)

array([[10480,    51,    25],
       [ 2713,  3828,   507],
       [ 1498,   218,  5650]], dtype=int64)

In [28]:
print(classification_report(df['sentiment'], all_predictions))

              precision    recall  f1-score   support

    negative       0.71      0.99      0.83     10556
     neutral       0.93      0.54      0.69      7048
    positive       0.91      0.77      0.83      7366

    accuracy                           0.80     24970
   macro avg       0.85      0.77      0.78     24970
weighted avg       0.83      0.80      0.79     24970

